__Random Monkey Stock Picker__

In [3]:
# load a list of all S&P 500 companies
import pandas as pd

df = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = df[0]
df = df[['Symbol', 'Security', 'GICS Sector']]
df['Ticker'] = df['Symbol']
df['Company'] = df['Security']
df = df[['Ticker', 'Company', 'GICS Sector']]
df.head()

,Ticker,Company,GICS Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott,Health Care
3,ABBV,AbbVie,Health Care
4,ACN,Accenture,Information Technology


Generate a random stock picker

In [50]:
import pandas as pd
import numpy as np

stock_data = pd.read_csv('SP500_stock_prices.csv')
stock_data['Date'] = pd.to_datetime(stock_data['Date'])
stock_data = stock_data[stock_data['Date'] >= '2019-07-05']
stock_data = stock_data[stock_data['Date'] <= '2020-04-02']
stock_data = stock_data[['Date', 'Ticker', 'Open']]
# Make Date the index
stock_data = stock_data.set_index('Date')

# Calculate returns over 63 business days
# select only the ticker index
indiviual_datas = []

for ticker in stock_data['Ticker'].unique():
    #data_ticker = data.xs(0, level=1).copy()
    data_ticker = stock_data[stock_data['Ticker'] == ticker].copy()
    # make target column
    data_ticker['63-day Price Movement'] = data_ticker['Open'].pct_change(62).shift(-62)
    #data['Price Movement 62 Days'] = data['Price Movement'].shift(62)
    data_ticker.dropna(inplace=True)
    data_ticker

    indiviual_datas.append(data_ticker)

data_all_tickers = pd.concat(indiviual_datas)
data_all_tickers

,Ticker,Open,63-day Price Movement
Date,,,
2019-07-05,MMM,173.000000,-0.092139
2019-07-08,MMM,169.919998,-0.086982
2019-07-09,MMM,166.820007,-0.069476
2019-07-10,MMM,166.190002,-0.067333
2019-07-11,MMM,166.300003,-0.084005
...,...,...,...
2019-12-26,ZTS,133.190002,-0.177791
2019-12-27,ZTS,133.399994,-0.141529
2019-12-30,ZTS,133.570007,-0.149285


In [51]:

# Set a seed for reproducibility
np.random.seed(0)
# Make a new Dataframe for the random portfolio returns
random_portfolio_returns = pd.DataFrame(columns=['Random Monkey - 3 Month Return (%)'])
# select 10 random tickers from the list for each date
for date in data_all_tickers.index.unique():
    random_stocks = np.random.choice(data_all_tickers['Ticker'].unique(), size=10, replace=False)

    # calculate the 63 day returns of the randomly selected stocks
    random_returns = data_all_tickers[data_all_tickers['Ticker'].isin(random_stocks)]['63-day Price Movement'].loc[date]

    # calculate the mean return of the random portfolio (equal weights)
    random_portfolio_return = random_returns.mean()
    # Make a new Dataframe for the random portfolio returns
    random_portfolio_returns.loc[date] = round(random_portfolio_return*100,1)

In [54]:
random_portfolio_returns
# save as a csv
random_portfolio_returns.to_csv('random_monkey_portfolio_returns.csv')